In [ ]:
import os
import random

import torch.nn as nn
import torch.cuda.amp as amp
import torch.utils.data
from tqdm import tqdm

from dataset import SRDataset
from loss import PerceptionLoss
from models import Generator, Discriminator
from utils import init_torch_seeds, convert_image

from PIL import PngImagePlugin
PngImagePlugin.MAX_TEXT_CHUNK = 100 * (1024**2)

Почти все параметры обучения идентичны тем, которые были использованы при обучении SRResNet на прошлом шаге.

In [ ]:
# параметры датасета
augments = {
    'rotation': False,
    'hflip' : True
}
crop_size = 256
lr_img_type = 'imagenet-norm'
hr_img_type = '[-1, 1]'
train_data_name = './jsons/train_images.json'

# параметры обучения модели
save_every = 30
print_every = 143
start_epoch = 0
iters = 1e5
batch_size = 24
lr = 1e-4
perception_loss_modifier = 0.06
beta = 1e-3 # модификатор adversarial ошибки
manualSeed = None
workers = 12

# параметры структуры модели
srresnet_checkpoint = './weights/SRResNet_16blocks_4x.pth' # путь к весам SRResNet, обученной на предыдущем этапе
upscale_factor = 4
n_blocks = 16

# Зададим рандомный seed, чтобы была возможность воспроизвести результат
if manualSeed is None:
    manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
init_torch_seeds(manualSeed)

Создаем SRDataset и dataloader

In [ ]:
dataset = SRDataset(crop_size=crop_size, scaling_factor=upscale_factor,
                    lr_img_type=lr_img_type, hr_img_type=hr_img_type,
                    train_data_name=train_data_name, augments=augments)

dataloader = torch.utils.data.DataLoader(dataset, shuffle=True,
                                         batch_size=batch_size,
                                         pin_memory=True,
                                         num_workers=int(workers))

Создадим генератор и дискриминатор, функции ошибки, оптимизаторы для генератора и дискриминатора (будем использовать Adam).<br>
В качестве функции ошибки мы используем взвешенную сумму **PerceptionLoss** и **BCEWithLogitsLoss**. <br>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# создаем генератор и дискриминатор
generator = Generator(n_blocks=n_blocks, scaling_factor=upscale_factor).to(device)
generator.load_state_dict(torch.load(srresnet_checkpoint)) # инициализируем модель весами srresnet
discriminator = Discriminator().to(device)

# инициализируем loss-ы
perception_criterion = PerceptionLoss().to(device) # MSE в пространстве фичей vgg19
adversarial_criterion = nn.BCEWithLogitsLoss().to(device)

# переводим в режим обучения
generator.train()
discriminator.train()

epochs = int(iters // len(dataloader))
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.9, 0.999))
optimizer_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.9, 0.999))

scaler_g = amp.GradScaler()
scaler_d = amp.GradScaler()

Запускаем обучение!

In [ ]:
for epoch in range(start_epoch, epochs):
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    g_avg_loss = 0.0
    d_avg_loss = 0.0
    for i, (lr_imgs, hr_imgs) in progress_bar:
        lr_imgs = lr_imgs.to(device)
        hr_imgs = hr_imgs.to(device)
        
        ### сначала обновляем дискриминатор
        optimizer_d.zero_grad()
        
        with amp.autocast():
            sr_imgs = generator(lr_imgs)
            
            hr_labels = discriminator(hr_imgs)
            fake_labels = discriminator(sr_imgs.detach())
            
            # Binary Cross-Entropy loss
            d_loss = adversarial_criterion(fake_labels, torch.zeros_like(fake_labels)) + \
                               adversarial_criterion(hr_labels, torch.ones_like(hr_labels))
        
        # back propagation
        scaler_d.scale(d_loss).backward()
        scaler_d.step(optimizer_d)
        scaler_d.update()
        
        ### обновляем генератор
        optimizer_g.zero_grad()
        
        with amp.autocast():
            # получаем fake high res изображения
            sr_imgs = generator(lr_imgs)
            
            # предсказания дискриминатора на фейковых изображениях
            fake_labels = discriminator(sr_imgs)

            # считаем loss-ы
            perception_loss = perception_loss_modifier * perception_criterion(sr_imgs, hr_imgs.detach())
            adversarial_loss = beta * adversarial_criterion(fake_labels, torch.ones_like(fake_labels))
            g_loss = perception_loss + adversarial_loss

        # back propagation
        scaler_g.scale(g_loss).backward()
        scaler_g.step(optimizer_g)
        scaler_g.update()

        d_avg_loss += d_loss.item()
        g_avg_loss += g_loss.item()

        progress_bar.set_description(f"[{epoch + 1}/{epochs}][{i + 1}/{len(dataloader)}] "
                                     f"Loss_D: {d_loss.item():.4f} Loss_G: {g_loss.item():.4f} ")

        total_iter = len(dataloader) * epoch + i
        
        if i % print_every == 0 and i != 0:
            print(f"Avg Loss_G: {(g_avg_loss/(i+1)):.4f} Avg Loss_D: {(d_avg_loss/(i+1)):.4f}")

            
    # сохраняем модели
    if (epoch+1)%save_every == 0:
        torch.save(generator.state_dict(), 
                   f"./weights/SRGAN_{n_blocks}blocks_{upscale_factor}x_epoch{(epoch+1)}.pth")
    else:
        torch.save(generator.state_dict(), f"./weights/SRGAN_{n_blocks}blocks_{upscale_factor}x.pth")